In [12]:
import io, os
from numpy import random
from google.cloud import vision
from PIL import ImageTk, Image, ImageDraw
import pandas as pd
import sqlite3

In [27]:
# create client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="static/cloud_credentials_steel-operative-337913-b3f7be97f534.json"
client = vision.ImageAnnotatorClient()

In [28]:
img_path = "static/images/paintings/"
paintings = os.listdir(img_path)

In [29]:
con = sqlite3.connect('rijksstudio.db')
cur = con.cursor()

In [30]:
# create objects table
cur.execute("""CREATE TABLE IF NOT EXISTS Objects
               (painting_id INTEGER PRIMARY KEY,
                object_name TEXT,
                confidence INTEGER,
                min_x INTEGER, 
                min_y INTEGER,
                max_x INTEGER, 
                max_y INTEGER,
                source TEXT
                )""")

# Save changes
con.commit()

In [39]:
def get_object_data(obj, painting):
    print(obj, painting)
    normalized_vertices = obj.bounding_poly.normalized_vertices
    q = cur.execute(f"""INSERT OR REPLACE INTO Objects
                           VALUES ({painting}, '{obj.name}', '{obj.score}, {normalized_vertices[0].x}, {normalized_vertices[0].y}, {normalized_vertices[1].x}, {normalized_vertices[1].y}, {"GOOGLE_VISION"}')
                           """)
    con.commit()
    

In [ ]:
for painting in paintings:
    print("Reading "+painting+" ...")
    # Loads the image into memory
    with io.open(os.path.join(img_path, painting), 'rb') as image_file:
        content = image_file.read()
    
    # call API
    image = vision.Image(content=content)
    response = client.object_localization(image=image)
    localized_object_annotations = response.localized_object_annotations
        
    # add every object data to db
    for obj in localized_object_annotations:
        # only use unique objects (for now)
        if obj not in objects:
            objects.append(obj)
            get_object_data(obj, painting)
    print("objects "+str(objects)+" added to db")

Reading SK-A-2152.png ...


In [ ]:
con.close()